In [2]:
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from arcgis.features import FeatureLayer

mhm_url = "https://services4.arcgis.com/feSRYIeirl5nUF92/arcgis/rest/services/MHM_2020/FeatureServer/0"

layer = FeatureLayer(mhm_url)

df = layer.query(where = "protocol = 'mosquito_habitat_mapper'").sdf
data = pd.DataFrame()

@interact(threshold = (0, 250))
def get_groups(threshold = 10):
    global df
    global data
    # groups / filters suspected events
    suspect_df = df.groupby(by=['measuredDate','latitude','mosquitohabitatmapperWaterSou_2','siteName','longitude']).filter(lambda x: len(x) > threshold)

    # identify groups
    suspect_groups = suspect_df.groupby(by = ['measuredDate','latitude','mosquitohabitatmapperWaterSou_2','siteName','longitude'])

    # create a groups dataset
    groups_data = pd.DataFrame()
    group_num = 1 
    for group, temp in suspect_groups:
        new_df = temp.copy(True)
        new_df["Group Data"] = str(group)
        new_df["Group Name"] = str(group_num)
        
        group_num += 1
        groups_data = groups_data.append(new_df, ignore_index = True)
        
    data = groups_data
    return groups_data


interactive(children=(IntSlider(value=10, description='threshold', max=250), Output()), _dom_classes=('widget-…

In [1]:
from arcgis.gis import GIS

gis = GIS( url="https://igestrategies.maps.arcgis.com", username="Piphi5")

Enter password:  ·········


In [5]:

output_file = "groups.geojson"

In [57]:
from arcgis.features import FeatureLayerCollection
import geopandas as gpd



geojson_data = data.drop(columns = ["SHAPE"])
gdf = gpd.GeoDataFrame(
    geojson_data, geometry=gpd.points_from_xy(geojson_data.longitude, geojson_data.latitude))

gdf.mosquitohabitatmapperComments .replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
gdf.to_file(output_file, driver='GeoJSON')

geojson_properties = {'overwrite':True}

groups_layer_id = "4971cba8fc7c48d3893d9bfdd9b6eecc"
group_layer_item = gis.content.get(groups_layer_id)


flc = FeatureLayerCollection.fromitem(group_layer_item)
flc.manager.overwrite(output_file)

#outcome = OverwriteFS.overwriteFeatureService(group_layer_item, "./mhm_groups.geojson")



AttributeError: 'NoneType' object has no attribute 'type'

In [6]:
mhm_geojson_properties = {'title' :'MHM Groups', 'type' : 'GeoJson', 'overwrite':'True', 'description' : '<p>MHM Data from GLOBE API URL: </p><p>Data is updated daily at 00:01 UTC</p><p><strong>Modification:</strong> GeoJSON file has been created from GLOBE API "non-GeoJSON" output. Coordinate geometry defined by mosquitohabitatmapperMeasurementLatitude and mosquitohabitatmappermeasurementLongitude coordinates, rather than the MGRS coordinates used in the GeoJSON generated by the GLOBE API</p>', 'tags' : 'MHM, GLOBE Observer Geospatial Data Hub, Modified'}
mhm_geojson_item = gis.content.add(item_properties=mhm_geojson_properties, data=output_file)


In [7]:
new_item = mhm_geojson_item.id
mhm_feature_layer_item = gis.content.get(new_item)
mhm_feature_layer_item.publish(overwrite = True)

Exception: Job failed.

In [17]:
from IPython.display import display, clear_output
import ipywidgets as widgets

from arcgis.features import SpatialDataFrame
from arcgis.geoenrichment import enrich
from arcgis.raster import ImageryLayer
from arcgis.geometry import Geometry
from arcgis.gis import GIS

groups_layer = group_layer_item.layers[0]
groups_layer_df = groups_layer.query(where = "protocol = 'mosquito_habitat_mapper'").sdf

# Create & Display Map
viz_map = gis.map('MHM Groups')
viz_map.add_layer(groups_layer)
viz_map.basemap = 'dark-gray-vector'
display(viz_map)

MapView(layout=Layout(height='400px', width='100%'))

In [7]:
import os
import glob
import re
import requests
import sys

def download_picture(url, directory):
    def get_picture(file_name):
        downloaded_obj = requests.get(url, allow_redirects=True)
        parent_dir = os.path.join(directory, file_name)
        with open(parent_dir, "wb") as file:
            file.write(downloaded_obj.content)

    if "https://" in url:
        photo_id = re.search(r'(?<=\d\d\d\d\/\d\d\/\d\d\/).*(?=\/)', url).group(0)
        file_name = f"{photo_id}.jpg".replace(":","-")
        get_picture(file_name)

        
def get_pictures(group):
    files = glob.glob('Images/*')
    for f in files:
        os.remove(f)

    group_water_source = group["mosquitohabitatmapperWaterSou_1"].dropna().tolist()
    group_abdomen = group["mosquitohabitatmapperAbdomenClo"].dropna().tolist()
    group_larvae = group["mosquitohabitatmapperLarvaFullB"].dropna().tolist()
    group_urls = group_water_source + group_abdomen + group_larvae
    dl = 0
    total_length = len(group_urls)
    for urls in group_urls:
        if urls:
            dl += 1
            urls = urls.split(";")
            for url in urls:
                download_picture(url, "Images")
        
        done = int(50 * dl / total_length)
        sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )    
        sys.stdout.flush()

group_name = ""
@interact
def select_group(group = groups_layer_df["Group_Name"].unique()):
    global group_name 
    group_name = group
    group_data = groups_layer_df[groups_layer_df["Group_Name"] == group]
    latitude = group_data["latitude"].iloc[0]
    longitude = group_data["longitude"].iloc[0]
    viz_map.center = [latitude, longitude]

button = widgets.Button(description='Get Photos')
out = widgets.Output()

def download_photos(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        print("Downloading Photos:")
        get_pictures(groups_layer_df[groups_layer_df["Group_Name"] == group_name])
        print(' Finished Downloading!')
# linking button and function together using a button's method
button.on_click(download_photos)
# displaying button and its output together
widgets.VBox([button,out])

interactive(children=(Dropdown(description='group', options=(1, 2, 3, 4, 5), value=1), Output()), _dom_classes…

In [114]:
import os
from IPython.display import Image
@interact
def show_images(file=os.listdir('Images/')):
    display(Image("Images/" + file))

interactive(children=(Dropdown(description='file', options=('1861016.jpg', '1861017.jpg', '1861018.jpg', '1861…